# Análisis Exploratorio de los datos de Trocafone



### Propuestas


## Configuracion inicial

In [ ]:
#Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('default')

In [ ]:
#Cargo el DataFrame
df = pd.read_csv('events.csv', low_memory=False)
df.head(5)

## Chequeos de integridad y calidad de los datos


In [ ]:
#Arreglo las fechas
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp'].head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
#Categorizo ciertas columnas para ahorrar espacio en memoria
for columna  in ['event','condition','storage', 'color', 'staticpage', 'campaign_source', 'search_engine', 'channel', 'new_vs_returning', 'region', 'country', 'device_type']:
        df[columna] = df[columna].astype('category')
df.info()


Como podemos ver, todas las filas especifican 'timestamp', 'event' y 'person'

## Analisis tipos de evento

In [ ]:
colUsadasEventos = df.groupby(by = 'event').count()
colUsadasEventos

In [ ]:
columns = list(df)
for event, row in colUsadasEventos.iterrows():
    print(event, end = ':')
    
    for column in columns:
        if(row.get(column) != 0):
            print(' ' + column, end = ',')
            
    print('\n')

## Columnas usadas por cada evento

Todos los eventos contienen informacion sobre **'timestamp'**, **'event'** y **'person'**, y ademas utilizan las siguientes columnas:

**ad campaign hit:**
    url, campaign_source

**brand listing:**
    skus

**checkout:** 
    sku, model, condition, storage, color

**conversion:**
    sku, model, condition, storage, color

**generic listing:**
    skus

**lead:**
    model

**search engine hit:** 
    search_engine

**searched products:** 
    skus, search_term

**staticpage:** 
    staticpage

**viewed product:** 
    sku, model, condition, storage, color

**visited site:** 
    channel, new_vs_returning, city, region, country, device_type, screen_resolution, operating_system_version, browser_version


## Analisis geográfico

In [ ]:
#Solo los eventos "visited site" especifican pais
visitas = df[df["event"] == "visited site"]

In [ ]:
visitas['country'].value_counts().head(10)

La mayor parte de las visitas provienen de Brasil, por lo que nos limitaremos a analizar a la audiencia brasileña

In [ ]:
visitas_brasil = visitas[visitas['country'] == 'Brazil']
visitas_brasil['region'].value_counts().head()

In [ ]:
#Comprobamos que no especifique ciudad para region de tipo "Unknown"
visitas_brasil[visitas_brasil['region'] == 'Unknown']['city'].value_counts().head()

In [ ]:
#Filtramos los 'Unknown'
visitas_brasil = visitas_brasil[visitas_brasil['region'] != 'Unknown']
visitas_brasil['region'].value_counts().head(15).plot('bar')

In [ ]:
visitas_brasil = visitas_brasil[visitas_brasil['city'] != 'Unknown']
visitas_brasil['city'].value_counts().head(15).plot('bar')

### Falta meterle un Basemap!!!!!!!!!!!!!!!!!!!

## Captacion de usuarios

In [ ]:
personas_new_ret = visitas.loc[:, ['person']]
personas_new_ret['returning_hits'] = visitas['new_vs_returning'] == 'Returning'
personas_new_ret.head()

In [ ]:
personas_ret_hits = personas_new_ret.groupby(by = 'person').agg({'returning_hits': np.count_nonzero})
personas_ret_hits.head(10)

In [ ]:
x = personas_ret_hits['returning_hits']
x.value_counts().head(10)

In [ ]:
sns.distplot(x, hist_kws={'log':True}, kde = False, axlabel = 'Cantidad de returning_hits').set_title('Cantidad de usuarios según returning_hits')

La mayoría de los usuarios ingresa al sitio una única vez

In [ ]:
cant_returns = x.value_counts()
cero_returns = cant_returns[0]
uno_diez_returns = cant_returns[1:11].values.sum()
diez_mas_returns = cant_returns[11:].values.sum()
datos_returns = [cero_returns, uno_diez_returns, diez_mas_returns]
x_labels = ['1', '2 - 10', '10+']
ax = sns.barplot(x = x_labels, y = datos_returns)
ax.set(xlabel='Cantidad de visitas al sitio', ylabel='Cantidad de Usuarios', title = 'Cantidad de usuarios segun veces que visitaron el sitio')

La mayoria de los usuarios visita el sitio una única vez, mientras que solo una pequeña proporción volvió al sitio mas de 10 veces. Es posible que un mismo usuario entre desde dispositivos diferentes.

## Analisis temporal

In [ ]:
#Vamos a hacer un analisis sobre los dias en los que se produjeron los eventos
dias_semana = df['timestamp'].dt.weekday_name
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_semana = pd.Categorical(dias_semana, categories = dias_ordenados, ordered = True)
g = dias_semana.value_counts().plot('bar')
g.set_title('Cantidad de eventos por dia de la semana')

Vemos que durante el fin de semana el trafico es menor

In [ ]:
tiempo_eventos = pd.DataFrame()
tiempo_eventos['dia_semana'] = df['timestamp'].dt.weekday_name
tiempo_eventos['hora'] = df['timestamp'].dt.hour
tiempo_eventos['mes'] = df['timestamp'].dt.month

tiempo_eventos = tiempo_eventos.groupby(by = 'dia_semana')['hora'].value_counts().to_frame()
tiempo_eventos = tiempo_eventos.rename(columns = {'hora':'cant_casos'})
tiempo_eventos = tiempo_eventos.reset_index('hora')
tiempo_eventos.index = pd.CategoricalIndex(tiempo_eventos.index, categories= reversed(dias_ordenados))
tiempo_eventos.sort_index(level=0, inplace=True)
tiempo_eventos = tiempo_eventos.reset_index()

tiempo_eventos = tiempo_eventos.pivot_table(index = 'dia_semana', columns = 'hora', values = 'cant_casos')

sns.set(rc={'figure.figsize':(19.7,8.27)})
cmap = sns.cm.rocket_r
ax = sns.heatmap(data = tiempo_eventos, cmap = cmap)
ax.set(xlabel='Hora', ylabel='Dia de la semana', title = 'Eventos segun hora del dia')

Como habiamos visto en el grafico anterior, durante el fin de semana la cantidad de eventos es menor, mientras que la mayoria de los eventos se realizan entre las 12 y las 2

In [ ]:
df_indice_en_persona_evento = df.set_index(['person','event'])
df_indice_en_persona_evento.head(50)

In [ ]:
def ver_diferencia_tiempo(tiempo):
    return tiempo.max()-tiempo.min()
    
diferencia_tiempo_entre_primero_y_ultimo_evento = df_indice_en_persona_evento.groupby('person').agg({'timestamp':ver_diferencia_tiempo})

In [ ]:
diferencia_tiempo_entre_primero_y_ultimo_evento.loc[diferencia_tiempo_entre_primero_y_ultimo_evento['timestamp']>'1 day'].sort_values(by = 'timestamp')

# Quiero ver Que campaña publicitaria lleva a mas ventas

In [ ]:
personas_por_publicidad = {}
    
ad_campaigns = df.loc[(df['event']=='ad campaign hit') | (df['event']=='conversion')].drop_duplicates()#[['person','campaign_source']].drop_duplicates().set_index('campaign_source')
ad_campaigns

In [ ]:
ad_campaigns_cool = df.loc[(df['event']=='ad campaign hit'),['person','campaign_source']].drop_duplicates()
ad_campaigns_cool.head()

In [ ]:
personas_que_compraron = df.loc[df['event']=='conversion',['person']].drop_duplicates()
personas_que_compraron['compro'] = True
personas_que_compraron.head()

In [ ]:
compras_por_campania = pd.merge(ad_campaigns_cool, personas_que_compraron, on='person', how='left')
compras_por_campania.head(10)

In [ ]:
compras2 = compras_por_campania.groupby('campaign_source').agg({'person': 'count','compro':'count'})
compras2.head()

In [ ]:
g = compras2['person'].sort_values(ascending=False).head().plot('bar')
g.set_title('Hits por campaña')
g.set_xlabel('Campaña publicitaria')
g.set_ylabel('Cantidad de hits')

In [ ]:
g2 = compras2['compro'].sort_values(ascending=False).plot('bar')
g2.set_title('Cantidad de compras realizadas por campaña')

In [ ]:
compras2['porcentaje'] = 100* (compras2['compro']/compras2['person'])
g3 = compras2['porcentaje'].sort_values(ascending=False).plot('bar')
g3.set_title('Porcentaje de compras logradas por campaña publicitaria')

Hay una "campaña publicitaria" llamada onsite que debería ser eliminada del data frame